In [56]:
import pandas as pd
import numpy as np
import os

from script import *

# Autoreload 
%load_ext autoreload
%autoreload 2

## Exploring number of images and labels

In [16]:
# Check number of training and test images
trainfiles = next(os.walk("../data/training_images"))[2]
print(f"Number of training images = {len(trainfiles)}")

testfiles = next(os.walk("../data/test_images"))[2]
print(f"Number of test images = {len(testfiles)}")


# Load annotation files
train_label = pd.read_csv("../data/training_elephants.csv", header=None)
train_label.columns = ['image', 'x', 'y']

test_label = pd.read_csv("../data/test_elephants.csv", header=None)
test_label.columns = ['image', 'x', 'y']


# Check number of training and test images with labeled elephants
print(
    f"Number of training images with elephants = {len(set(train_label.image))}")
print(f"Number of test images with elephants = {len(set(test_label.image))}")


# Check total number of elephants labeled in training and test images
print(f"Number of elephants in training images = {len(train_label)}")
print(f"Number of elephants in test images = {len(test_label)}")

Number of training images = 1635
Number of test images = 439
Number of training images with elephants = 1629
Number of test images with elephants = 438
Number of elephants in training images = 12611
Number of elephants in test images = 2970


In [57]:
check_duplicates(train_label)
check_duplicates(test_label)

You are all clear of duplicates
You are all clear of duplicates


## Exploring images metadata

In [23]:
# Load metadata files
train_metadata = pd.read_csv("../data/training_images.csv", header=None)
train_metadata.columns = ["image_name", "sortie_id", "width", "height", "GSD", "measured_altitude", "terrain_altitude", "GPS_Altitude"]

test_metadata = pd.read_csv("../data/test_images.csv", header=None)
test_metadata.columns = ["image_name", "sortie_id", "width", "height", "GSD", "measured_altitude", "terrain_altitude", "GPS_Altitude"]


>Different altitude measures:
>- Measured altitude was determined by an onboard laser altimeter, 
>- GPS altitude was measured by the camera gps 
>- Terrain altitude was obtained by using the GPS coordinates in concert with SRTM data.

In [26]:
train_metadata.describe().T

,count,mean,std,min,25%,50%,75%,max
sortie_id,1635.0,52.709480,12.505984,6.000000,53.000000,58.000000,59.000000,64.000000
width,1635.0,5487.392661,21.190630,5472.000000,5472.000000,5472.000000,5496.000000,5525.000000
height,1635.0,3660.733945,17.053363,3648.000000,3648.000000,3648.000000,3670.000000,3690.000000
GSD,1635.0,0.075041,0.031969,0.024298,0.043469,0.082256,0.090752,0.129874
measured_altitude,1635.0,1529.446972,703.890331,-1.000000,1541.500000,1825.000000,1867.450000,2269.600000
terrain_altitude,1635.0,733.638532,245.979641,0.000000,624.000000,743.000000,949.500000,1236.000000
GPS_Altitude,1635.0,1659.967768,513.202749,219.700000,1548.750000,1825.000000,1867.450000,2269.600000


In [27]:
test_metadata.describe().T

,count,mean,std,min,25%,50%,75%,max
sortie_id,439.0,52.448747,13.404201,2.000000,53.000000,58.000000,60.0000,64.00000
width,439.0,5484.920273,18.960919,5472.000000,5472.000000,5472.000000,5496.0000,5525.00000
height,439.0,3658.902050,15.464703,3648.000000,3648.000000,3648.000000,3670.0000,3690.00000
GSD,439.0,0.067933,0.028967,0.012327,0.043367,0.056509,0.0846,0.12951
measured_altitude,439.0,1413.498178,723.743467,0.000000,1517.850000,1656.500000,1861.5500,2263.10000
terrain_altitude,439.0,755.173121,263.825842,38.000000,626.000000,763.000000,961.0000,1229.00000
GPS_Altitude,439.0,1572.266970,511.043668,230.800000,1540.550000,1656.500000,1861.5500,2263.10000


In [58]:
zero_altitude = test_metadata.loc[test_metadata.measured_altitude == 0]
zero_images = zero_altitude.image_name

show_image(zero_images)

In [55]:
max_altitude = test_metadata.loc[test_metadata.measured_altitude == test_metadata.measured_altitude.max()]
max_images = max_altitude.image_name

show_image(max_images)

Images with extreme altitude measurements (e.g. 0 measured altitude, highest altitude measured) were inspected, and although it is apparent that the altitude form which the images were taken vary, they seem to be valid images for analysis. 

For future use:  
- min w = 5472  
- min h = 3648  

## Creating bounding boxes

The annotations provided with the dataset are in forma of an x, y pixel coordinate. Here we will attempt to create bounding boxes out of the information.

In [59]:
train_label.head()

,image,x,y
0,7c2b5fa7fefd29566bacc3fc14de73d35d64da6c,4294,845
1,7c2b5fa7fefd29566bacc3fc14de73d35d64da6c,4498,707
2,a46adbea1faf34cf3ecbad37886b33fb01fd8c63,2781,262
3,e67a5464acd5bcec0c6b84d0a99961a66981a2ea,626,898
4,e67a5464acd5bcec0c6b84d0a99961a66981a2ea,574,886


In [63]:
w=5
h=5

#Training labels 
train_label['xmin'] = np.array(train_label['x'])-w
train_label['xmax'] = np.array(train_label['x'])+w

train_label['ymin'] = np.array(train_label['y'])-h
train_label['ymax'] = np.array(train_label['y'])+h


#Test labels 
test_label['xmin'] = np.array(test_label['x'])-w
test_label['xmax'] = np.array(test_label['x'])+w

test_label['ymin'] = np.array(test_label['y'])-h
test_label['ymax'] = np.array(test_label['y'])+h


In [65]:
train_label.head()

,image,x,y,xmin,xmax,ymin,ymax
0,7c2b5fa7fefd29566bacc3fc14de73d35d64da6c,4294,845,4289,4299,840,850
1,7c2b5fa7fefd29566bacc3fc14de73d35d64da6c,4498,707,4493,4503,702,712
2,a46adbea1faf34cf3ecbad37886b33fb01fd8c63,2781,262,2776,2786,257,267
3,e67a5464acd5bcec0c6b84d0a99961a66981a2ea,626,898,621,631,893,903
4,e67a5464acd5bcec0c6b84d0a99961a66981a2ea,574,886,569,579,881,891
